In [34]:
import hail as hl
import pandas as pd
# Parameters
log_path = '../logs/hail'
file_in = '../data/merged.vcf.gz'
file_out = '../data/merged.mt'


In [ ]:
hl.init(app_name = 'gnomAD', log = log_path)

Loading BokehJS ...

25/10/31 22:21:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.5.7
SparkUI available at http://pimenta:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.136-c32f88309ab0
LOGGING: writing to ../logs/hail/


2025-10-31 22:21:30.525 Hail: INFO: scanning VCF for sortedness...
2025-10-31 22:21:32.466 Hail: INFO: Coerced sorted VCF - no additional import work to do
2025-10-31 22:21:34.076 Hail: INFO: wrote matrix table with 2000 rows and 0 columns in 1 partition to ../data/merged.mt
2025-10-31 22:31:02.810 Hail: INFO: wrote matrix table with 2000 rows and 0 columns in 1 partition to ../data/merged.mt
2025-10-31 22:44:40.151 Hail: INFO: merging 2 files totalling 458,3K...
2025-10-31 22:44:40.160 Hail: INFO: while writing:
    ../data/variant_stats.tsv
  merge time: 7,899ms


In [10]:
# Import and write matrix table
mt = hl.import_vcf(file_in, force_bgz=True, reference_genome='GRCh38')
mt.write(file_out, overwrite=True)

In [11]:
mt = hl.read_matrix_table(file_out)

In [23]:
# Here need to be changed in the real dataset. The current dataset is just a mockup.
# For example, adding annotations from gnomAD, filtering samples, etc.
# # Calculate basic allele statistics (similar to gnomAD)
# mt = hl.variant_qc(mt)
print("Matrix table schema:")
mt.describe()

Matrix table schema:
----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        AC: array<int32>, 
        AN: int32, 
        AF: array<float64>, 
        grpmax: array<str>, 
        fafmax_faf95_max: array<float64>, 
        fafmax_faf95_max_gen_anc: array<str>, 
        AC_XX: array<int32>, 
        AF_XX: array<float64>, 
        AN_XX: int32, 
        nhomalt_XX: array<int32>, 
        AC_XY: array<int32>, 
        AF_XY: array<float64>, 
        AN_XY: int32, 
        nhomalt_XY: array<int32>, 
        nhomalt: array<int32>, 
        AC_afr_XX: array<int32>, 
        AF_afr_XX: array<float64>, 
        AN_afr_XX: int32, 
        nhomalt_afr_XX: array<int32>, 
        AC_afr_XY: array<int32>, 
        AF_afr_XY: ar

In [29]:
# Extract gnomAD statistics from info field
variants = mt.rows()

In [30]:
# helper function to extract info field
def first_or_default(expr, default):
    return hl.if_else(hl.len(expr) > 0, expr[0], default)


In [ ]:
# Select relevant fields from the info struct
variants = variants.select(
    # Locus information
    chrom=variants.locus.contig,
    pos=variants.locus.position,
    ref=variants.alleles[0],
    alt=hl.delimit(variants.alleles[1:], ','),  # Join alt alleles if multiple
    rsid=variants.rsid,

    # Basic allele statistics (from gnomAD info field)
    AC=first_or_default(variants.info.AC, 0),
    AN=variants.info.AN,
    AF=first_or_default(variants.info.AF, 0.0),

    # Genetic ancestry group with max AF
    grpmax=first_or_default(variants.info.grpmax, ''),

    # Filtering Allele Frequency (95% confidence)
    faf95_max=first_or_default(variants.info.fafmax_faf95_max, 0.0),
    faf95_max_gen_anc=first_or_default(variants.info.fafmax_faf95_max_gen_anc, ''),

    # Number of homozygotes
    nhomalt=first_or_default(variants.info.nhomalt, 0),

    # Sex-specific counts
    AC_XX=first_or_default(variants.info.AC_XX, 0),
    AN_XX=variants.info.AN_XX,
    AF_XX=first_or_default(variants.info.AF_XX, 0.0),
    nhomalt_XX=first_or_default(variants.info.nhomalt_XX, 0),

    AC_XY=first_or_default(variants.info.AC_XY, 0),
    AN_XY=variants.info.AN_XY,
    AF_XY=first_or_default(variants.info.AF_XY, 0.0),
    nhomalt_XY=first_or_default(variants.info.nhomalt_XY, 0),

    # Population-specific (examples)
    AC_afr=first_or_default(variants.info.AC_afr, 0),
    AF_afr=first_or_default(variants.info.AF_afr, 0.0),
    AN_afr=variants.info.AN_afr,

    AC_amr=first_or_default(variants.info.AC_amr, 0),
    AF_amr=first_or_default(variants.info.AF_amr, 0.0),
    AN_amr=variants.info.AN_amr,

    AC_eas=first_or_default(variants.info.AC_eas, 0),
    AF_eas=first_or_default(variants.info.AF_eas, 0.0),
    AN_eas=variants.info.AN_eas,

    AC_nfe=first_or_default(variants.info.AC_nfe, 0),
    AF_nfe=first_or_default(variants.info.AF_nfe, 0.0),
    AN_nfe=variants.info.AN_nfe,

    AC_sas=first_or_default(variants.info.AC_sas, 0),
    AF_sas=first_or_default(variants.info.AF_sas, 0.0),
    AN_sas=variants.info.AN_sas,

    # Prediction scores
    cadd_phred=variants.info.cadd_phred,
    revel_max=variants.info.revel_max,
    spliceai_ds_max=variants.info.spliceai_ds_max,

    # Quality metrics
    qual=variants.qual,
    filters=hl.delimit(variants.filters, ','),
)


In [32]:
# Convert to pandas
df = variants.to_pandas()

print("\n=== Variant Statistics ===")
print(f"Total variants: {len(df):,}")
print("\nFirst few variants:")
print(df.head(10))

# Export to TSV for Streamlit
output_tsv = '../data/variant_stats.tsv'
variants.export(output_tsv)
print(f"\nExported to: {output_tsv}")

# Summary statistics
print("\n=== Summary Statistics ===")
print(f"Mean Allele Count: {df['AC'].mean():.2f}")
print(f"Mean Allele Frequency: {df['AF'].mean():.6f}")
print(f"Mean Allele Number: {df['AN'].mean():.2f}")
print(f"Total Homozygotes: {df['nhomalt'].sum():,}")
print(f"\nChromosomes present: {sorted(df['chrom'].unique())}")
print(f"Variants with rsID: {df['rsid'].notna().sum():,}")


=== Variant Statistics ===
Total variants: 2,000

First few variants:
          locus       alleles  chrom     pos      ref  alt          rsid  \
0   chr18:48494        [T, A]  chr18   48494        T    A  rs1905848822   
1   chr18:87087       [TG, T]  chr18   87087       TG    T          <NA>   
2  chr18:109438        [T, C]  chr18  109438        T    C  rs1462125550   
3  chr18:109567      [C, CCT]  chr18  109567        C  CCT          <NA>   
4  chr18:112506  [CCTCCAG, C]  chr18  112506  CCTCCAG    C          <NA>   
5  chr18:223630        [C, A]  chr18  223630        C    A          <NA>   
6  chr18:224952        [C, A]  chr18  224952        C    A          <NA>   
7  chr18:225235        [C, T]  chr18  225235        C    T  rs1911240230   
8  chr18:225321        [T, C]  chr18  225321        T    C   rs755941042   
9  chr18:260110        [A, G]  chr18  260110        A    G   rs540995383   

     AC    AN        AF  ...    AF_nfe   AN_nfe AC_sas  AF_sas  AN_sas  \
0  <NA>  <NA>  0.0